In [7]:
import requests
from bs4 import BeautifulSoup
import re #regular expressions

In [3]:
#Sending the request
request=requests.get('https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha')
print(request.status_code)

200
<Response [200]>


In [5]:
#Making a soup
soup=BeautifulSoup(request.content)
#Extracting restaurant titles
titles_soup=soup.find_all('h3')
titles=[]
for i in titles_soup:
    titles.append(i.find('a').text)

['Repre restaurant', 'Restaurace Nad Ledem', 'SRI LANKAN CURRY HOUSE', 'Restaurace HOOTERS Vodičkova', '1. MEXICKÁ RESTAURACE', 'A.H.W. a.s.', 'Abri s.r.o.', 'ABSOLUT s.r.o.', 'ADAMOVÁ IVA', 'Adeta Service, s.r.o.', 'Adiscon Kampa s.r.o.', 'ADMIRÁL - RESTAURACE', 'Aftermath, s.r.o.', 'AGIOS s.r.o.', 'AGUATERRA s.r.o.', 'AIKYO FOODS CZECH', 'Air Pizza, s.r.o.', 'Aishel, s.r.o.', 'Al Karim', 'Al-Mare Těšnov s. r. o.', 'ALBIO', 'ALDEN', 'ALDENTE TRATTORIA ITALIANA', 'Aleš Tománek', 'Alforno Focacceria Italiana - FINE FOOD CZ, s. r. o.', 'ALLEGRO', 'ALMA restaurace ALMA', 'ALMIRA MV, s.r.o.', 'ALOHA - MUSIC CLUB & COCKTAIL BAR PRAHA', 'Alois Filip', 'Alpská restaurace', 'Alriso, s.r.o.', 'Ambiente - The Living Restaurant', 'Ambiente restaurace', 'AMBROISIE CAFFÉ RESTAURANT', 'AMBROZIE', 'Amfora restaurant', 'AMI RESTAURANT s.r.o.', 'Amigos - restaurace s mexickou a českou kuchyní', 'AMOS RESTAURANT', 'AN QI INTERNATIONAL TRADING spol. s r.o.', 'Ananta vegetarian restaurant', 'ANDĚL APARTME

In [9]:
#Extracting the link for each restaurant
links=[]
for i in titles_soup:
    links.append(f'https://www.zlatestranky.cz{i.find("a")["href"]}')

['https://www.zlatestranky.cz/profil/H270008', 'https://www.zlatestranky.cz/profil/H2052564', 'https://www.zlatestranky.cz/profil/H2028609', 'https://www.zlatestranky.cz/profil/H2056923', 'https://www.zlatestranky.cz/profil/H952175', 'https://www.zlatestranky.cz/profil/T1488202', 'https://www.zlatestranky.cz/profil/H731735', 'https://www.zlatestranky.cz/profil/T678796', 'https://www.zlatestranky.cz/profil/H217233', 'https://www.zlatestranky.cz/profil/H1909419', 'https://www.zlatestranky.cz/profil/H1477102', 'https://www.zlatestranky.cz/profil/T745675', 'https://www.zlatestranky.cz/profil/T2784508', 'https://www.zlatestranky.cz/profil/H1365763', 'https://www.zlatestranky.cz/profil/H1205046', 'https://www.zlatestranky.cz/profil/H826759', 'https://www.zlatestranky.cz/profil/H1587208', 'https://www.zlatestranky.cz/profil/H1908905', 'https://www.zlatestranky.cz/profil/H2052606', 'https://www.zlatestranky.cz/profil/H2054309', 'https://www.zlatestranky.cz/profil/H168141', 'https://www.zlatest

In [5]:
#Request for a single restaurant page
request_profil=requests.get('https://www.zlatestranky.cz/profil/H270008')

<address itemprop="address" itemscope="" itemtype="https://schema.org/PostalAddress">
<meta content="Nekázanka 4/857" itemprop="streetAddress"/>
<meta content="Praha" itemprop="addressLocality"/>
<meta content="110 00" itemprop="postalCode"/>
<span itemprop="description">Nekázanka 4/857, 110 00 Praha 1-Nové Město<br/><small>okres Hlavní město Praha, Hlavní město Praha</small></span>
</address>

In [13]:
#Extracting the district 
profil_soup=BeautifulSoup(request_profil.content)
address=profil_soup.find('address').find('span').text
re.search('Praha [0-9]{1,2}',address).group(0)

'Praha 1'

In [15]:
#Extracting the ratings
profil_soup.find('span',{'itemprop':'ratingValue'}).text

'0'

In [16]:
#Extracting the number of reviews
profil_soup.find('span',{'itemprop':'reviewCount'}).text

'0'

In [31]:
### Extracting the opening hours ###
opening_hours_table=profil_soup.find('table',{'class':'table table-condensed'}).find_all('td') 
opening_hours_table_text=[] #empty list
#extract only day and hours
for i in opening_hours_table:
    if re.match('Dnes|\n',i.text):
        pass
    else:
        opening_hours_table_text.append(i.text)
print(opening_hours_table_text)
#creating a dictionary (key=day,value=opening hours)
opening_hours_dict={}
for i in range(0,len(opening_hours_table_text),2):
    opening_hours_dict[opening_hours_table_text[i]]=opening_hours_table_text[i+1]
print(opening_hours_dict)


['Po', '10:30 - 23', 'Út', '10:30 - 23', 'St', '10:30 - 23', 'Čt', '10:30 - 23', 'Pá', '10:30 - 23', 'So', '12 - 23']
{'Po': '10:30 - 23', 'Út': '10:30 - 23', 'St': '10:30 - 23', 'Čt': '10:30 - 23', 'Pá': '10:30 - 23', 'So': '12 - 23'}


In [48]:
### Calculating the opening time span for each day ###
#Writing a function to convert the time span into a number
def time_span_to_number(time_span):
    hours=[time_span.split()[i] for i in [0,2]] #splitting by space and disregarding "-"
    start_end=[] #empty list for converted starting and ending values
    for i in hours:
        if re.search(':',i): #if the string is not a whole hour, convert it
            splits=i.split(':')
            start_end.append(float(splits[0])+float(splits[1])/60)
        else:
            start_end.append(float(i))
    return round(start_end[1] - start_end[0],2)

time_span_dict={key:time_span_to_number(value) for (key,value) in opening_hours_dict.items()}
print(time_span_dict)

{'Po': 12.5, 'Út': 12.5, 'St': 12.5, 'Čt': 12.5, 'Pá': 12.5, 'So': 11.0}


In [ ]:
#requests for all pages !!! NOT TESTED YET !!!
request_list=[]
i=1
while request.status_code == 200:
    request=requests.get(f'https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha/{i}')
    if request.status_code == 200:
        request_list.append(request.content)
        i += 1
    else:
        print(f'Successfuly requested {i-1} pages')
        